In [1]:
import numpy as np

from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp

In [2]:
num_layers = 4
n_qubits = 3

inputs = ParameterVector("input", n_qubits)
weights = ParameterVector("weights", num_layers * n_qubits)

qc = QuantumCircuit(3)
for i in range(n_qubits):
    qc.rx(inputs[i], i)

for i in range(n_qubits):
    qc.ry(weights[i], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(weights[i+3], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rx(weights[i+6], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(weights[i+9], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

In [3]:
qc.draw()

┌──────────────┐┌────────────────┐          ┌───┐┌────────────────┐     »
q_0: ┤ Rx(input[0]) ├┤ Ry(weights[0]) ├──■───────┤ X ├┤ Rz(weights[3]) ├──■──»
     ├──────────────┤├────────────────┤┌─┴─┐     └─┬─┘├────────────────┤┌─┴─┐»
q_1: ┤ Rx(input[1]) ├┤ Ry(weights[1]) ├┤ X ├──■────┼──┤ Rz(weights[4]) ├┤ X ├»
     ├──────────────┤├────────────────┤└───┘┌─┴─┐  │  ├────────────────┤└───┘»
q_2: ┤ Rx(input[2]) ├┤ Ry(weights[2]) ├─────┤ X ├──■──┤ Rz(weights[5]) ├─────»
     └──────────────┘└────────────────┘     └───┘     └────────────────┘     »
«          ┌───┐┌────────────────┐          ┌───┐ ┌────────────────┐          »
«q_0: ─────┤ X ├┤ Rx(weights[6]) ├──■───────┤ X ├─┤ Rz(weights[9]) ├──■───────»
«          └─┬─┘├────────────────┤┌─┴─┐     └─┬─┘┌┴────────────────┤┌─┴─┐     »
«q_1: ──■────┼──┤ Rx(weights[7]) ├┤ X ├──■────┼──┤ Rz(weights[10]) ├┤ X ├──■──»
«     ┌─┴─┐  │  ├────────────────┤└───┘┌─┴─┐  │  ├─────────────────┤└───┘┌─┴─┐»
«q_2: ┤ X ├──■──┤ Rx(weights[8]) ├─────┤ X ├──■──┤ Rz(weights[11]) ├─────┤ X ├»
«     └───┘     └────────────────┘     └───┘     └─────────────────┘     └───┘»
«     ┌───┐
«q_0: ┤ X ├
«     └─┬─┘
«q_1: ──┼──
«       │  
«q_2: ──■──
«

In [4]:
obs = SparsePauliOp("ZZI")

In [5]:
service = QiskitRuntimeService()
backend = service.get_backend("ibmq_qasm_simulator")

In [6]:
weights_init = np.random.randn(num_layers * n_qubits)
inputs_init = np.random.randn(n_qubits)

In [7]:
qc.assign_parameters({weights:weights_init, inputs:inputs_init}, inplace=True)

In [8]:
qc.draw()

┌────────────┐ ┌─────────────┐           ┌───┐┌─────────────┐           »
q_0: ─┤ Rx(-1.416) ├─┤ Ry(-1.6166) ├───■───────┤ X ├┤ Rz(0.46169) ├───■───────»
     ┌┴────────────┴┐├─────────────┴┐┌─┴─┐     └─┬─┘├─────────────┴┐┌─┴─┐     »
q_1: ┤ Rx(-0.24067) ├┤ Ry(-0.11984) ├┤ X ├──■────┼──┤ Rz(-0.50251) ├┤ X ├──■──»
     ├─────────────┬┘├─────────────┬┘└───┘┌─┴─┐  │  ├─────────────┬┘└───┘┌─┴─┐»
q_2: ┤ Rx(0.37597) ├─┤ Ry(-1.5287) ├──────┤ X ├──■──┤ Rz(-1.0869) ├──────┤ X ├»
     └─────────────┘ └─────────────┘      └───┘     └─────────────┘      └───┘»
«     ┌───┐┌──────────────┐          ┌───┐ ┌─────────────┐           ┌───┐
«q_0: ┤ X ├┤ Rx(-0.59304) ├──■───────┤ X ├─┤ Rz(0.58515) ├───■───────┤ X ├
«     └─┬─┘├──────────────┤┌─┴─┐     └─┬─┘ ├─────────────┴┐┌─┴─┐     └─┬─┘
«q_1: ──┼──┤ Rx(-0.13782) ├┤ X ├──■────┼───┤ Rz(-0.69088) ├┤ X ├──■────┼──
«       │  ├─────────────┬┘└───┘┌─┴─┐  │  ┌┴──────────────┤└───┘┌─┴─┐  │  
«q_2: ──■──┤ Rx(-1.1636) ├──────┤ X ├──■──┤ Rz(-0.021717) ├─────┤ X ├──■──
«          └─────────────┘      └───┘     └───────────────┘     └───┘

In [9]:
estimator = Estimator(backend)

In [10]:
job = estimator.run([qc, qc], observables=[obs, obs], shots=1024)

In [11]:
result = job.result()

In [12]:
result

EstimatorResult(values=array([0.03515625, 0.046875  ]), metadata=[{'variance': 0.9987640380859375, 'shots': 1024}, {'variance': 0.997802734375, 'shots': 1024}])